In [2]:
import numpy as np
import pandas as pd
import os
import re
import emoji

In [3]:
train_df = pd.read_csv('Data/train.csv')
test_df = pd.read_csv('Data/test.csv')
sampledf = pd.read_csv('Data/sample_submission.csv')

In [4]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r',', '', text)  # Remove commas
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    return text

# Apply the clean_text function to the 'text' column
train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)



In [5]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,Heard about earthquake is different cities sta...
2,3,NaN,NaN,there is a forest fire at spot pond geese are ...
3,9,NaN,NaN,Apocalypse lighting Spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
train_df.shape

(7613, 5)

In [7]:
from transformers import AutoTokenizer, TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert = TFBertModel.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

c:\Users\Soham R\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:1295: UserWarning: Layer 'tf_bert_model' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''The following keyword arguments are not supported by this model: ['kwargs'].''
  warnings.warn(
c:\Users\Soham R\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()` was called on layer 'tf_bert_model', however the

In [8]:
print("max len of tweet", max([len(x.split()) for x in train_df.text]))


max len of tweet 31


In [9]:
max_len = 34

In [10]:
x_train = tokenizer(
    text = train_df.text.tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation= True,
    padding = True,
    return_tensors = 'tf',
    return_token_type_ids= False,
    return_attention_mask=True,
    verbose= True)

In [11]:
x_train['input_ids'].shape
x_train

{'input_ids': <tf.Tensor: shape=(7613, 34), dtype=int32, numpy=
array([[  101,  2256, 15616, ...,     0,     0,     0],
       [  101,  3224,  2543, ...,     0,     0,     0],
       [  101,  2035,  3901, ...,     0,     0,     0],
       ...,
       [  101, 23290,  2683, ...,     0,     0,     0],
       [  101,  2610, 11538, ...,     0,     0,     0],
       [  101,  1996,  6745, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(7613, 34), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

In [12]:
x_train['attention_mask'].shape

TensorShape([7613, 34])

In [13]:
y_train = train_df.target.values
y_train

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [14]:
# Assuming x_train is your tokenized input from earlier


In [15]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.initializers import TruncatedNormal
from keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from keras.metrics import CategoricalAccuracy,BinaryAccuracy
from keras.utils import to_categorical
from keras.utils import plot_model

In [16]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [17]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": x_train["input_ids"],
        "attention_mask": x_train["attention_mask"]
    },
    train_df.target.values
))

# Batch and prefetch for performance
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
from transformers import TFBertModel

# Load pre-trained BERT model
bert = TFBertModel.from_pretrained('bert-large-uncased')

# Define the model as a tf.keras.Model subclass
class BertClassifier(tf.keras.Model):
    def __init__(self, bert_model):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.dropout1 = tf.keras.layers.Dropout(0.1)
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.1)
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, training=training)
        pooled_output = bert_output.pooler_output
        x = self.dropout1(pooled_output, training=training)
        x = self.dense1(x)
        x = self.dropout2(x, training=training)
        x = self.dense2(x)
        return self.output_layer(x)

# Instantiate the model
model = BertClassifier(bert)

# Define loss function and optimizer
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

# Custom accuracy function
def binary_accuracy(y_true, y_pred):
    y_pred = tf.round(y_pred)
    return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred), tf.float32))

# Define training step
@tf.function
def train_step(input_ids, attention_mask, labels):
    with tf.GradientTape() as tape:
        predictions = model((input_ids, attention_mask), training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    accuracy = binary_accuracy(labels, predictions)
    return loss, accuracy

# Prepare your data
# Assuming x_train is your tokenized input from earlier
input_ids = tf.convert_to_tensor(x_train["input_ids"], dtype=tf.int32)
attention_mask = tf.convert_to_tensor(x_train["attention_mask"], dtype=tf.int32)
labels = tf.convert_to_tensor(train_df.target.values, dtype=tf.float32)

# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels))
dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(32)

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    
    for batch in dataset:
        input_ids_batch, attention_mask_batch, labels_batch = batch
        loss, accuracy = train_step(input_ids_batch, attention_mask_batch, labels_batch)
        total_loss += loss
        total_accuracy += accuracy
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches
    
    print(f'Epoch {epoch + 1}, '
          f'Loss: {avg_loss:.4f}, '
          f'Accuracy: {avg_accuracy:.4f}')

In [46]:
# import tensorflow as tf
# from transformers import TFBertModel
# from tqdm import tqdm
# # Load pre-trained BERT model


# # Define the model as a tf.keras.Model subclass
# class BertClassifier(tf.keras.Model):
#     def __init__(self, bert_model):
#         super(BertClassifier, self).__init__()
#         self.bert = bert_model
#         self.dropout1 = tf.keras.layers.Dropout(0.1)
#         self.dense1 = tf.keras.layers.Dense(128, activation='relu')
#         self.dropout2 = tf.keras.layers.Dropout(0.1)
#         self.dense2 = tf.keras.layers.Dense(32, activation='relu')
#         self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

#     def call(self, inputs, training=False):
#         input_ids, attention_mask = inputs
#         bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, training=training)
#         pooled_output = bert_output.pooler_output
#         x = self.dropout1(pooled_output, training=training)
#         x = self.dense1(x)
#         x = self.dropout2(x, training=training)
#         x = self.dense2(x)
#         return self.output_layer(x)

# # Instantiate the model
# model = BertClassifier(bert)

# # Define loss function and optimizer
# loss_object = tf.keras.losses.BinaryCrossentropy()
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

# # Custom accuracy function
# def binary_accuracy(y_true, y_pred):
#     y_pred = tf.round(y_pred)
#     return tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred), tf.float32))

# # Define training step
# @tf.function
# def train_step(input_ids, attention_mask, labels):
#     with tf.GradientTape() as tape:
#         predictions = model((input_ids, attention_mask), training=True)
#         loss = loss_object(labels, predictions)
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     accuracy = binary_accuracy(labels, predictions)
#     return loss, accuracy

# # Prepare your data
# # Assuming x_train is your tokenized input from earlier
# input_ids = tf.convert_to_tensor(x_train["input_ids"], dtype=tf.int32)
# attention_mask = tf.convert_to_tensor(x_train["attention_mask"], dtype=tf.int32)
# labels = tf.convert_to_tensor(train_df.target.values, dtype=tf.float32)

# # Create tf.data.Dataset
# dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels))
# dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(32)

# # Training loop
# # Training loop
# epochs = 10
# best_accuracy = 0
# save_path = 'saved_model'

# # Create a progress bar for epochs
# epoch_progress_bar = tqdm(range(epochs), desc="Epochs", position=0)

# for epoch in epoch_progress_bar:
#     total_loss = 0
#     total_accuracy = 0
#     num_batches = 0
    
#     # Create a progress bar for batches
#     batch_progress_bar = tqdm(dataset, desc=f"Epoch {epoch+1}", leave=False, position=1)
    
#     for batch in batch_progress_bar:
#         input_ids_batch, attention_mask_batch, labels_batch = batch
#         loss, accuracy = train_step(input_ids_batch, attention_mask_batch, labels_batch)
#         total_loss += loss
#         total_accuracy += accuracy
#         num_batches += 1
        
#         # Update batch progress bar
#         batch_progress_bar.set_postfix({"Loss": f"{loss:.4f}", "Accuracy": f"{accuracy:.4f}"})
    
#     avg_loss = total_loss / num_batches
#     avg_accuracy = total_accuracy / num_batches
    
#     # Update epoch progress bar
#     epoch_progress_bar.set_postfix({"Avg Loss": f"{avg_loss:.4f}", "Avg Accuracy": f"{avg_accuracy:.4f}"})
    
#     # Save the model if it's the best so far
#     if avg_accuracy > best_accuracy:
#         best_accuracy = avg_accuracy
        
#         # Save the entire model
#         tf.keras.models.save_model(model, save_path)
#         print(f"\nModel saved to {save_path}")

# print("\nTraining completed. Best model saved.")

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epochs:   0%|          | 0/10 [01:36<?, ?it/s]


KeyboardInterrupt: 

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[1] #(0 is the last hidden states,1 means pooler_output)
# out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = tf.keras.layers.Dropout(0.1)(embeddings)

out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(1,activation = 'sigmoid')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

ValueError: Exception encountered when calling TFBertModel.call().

[1mAll operation outputs must be tensors. Operation <TFBertModel name=tf_bert_model, built=True> returned a non-tensor. Non-tensor received: TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor 'bert_1/encoder_1/layer_._23_1/output_1/LayerNorm_1/add_2:0' shape=(None, 34, 1024) dtype=float32>, pooler_output=<tf.Tensor 'bert_1/pooler_1/dense_1/Tanh:0' shape=(None, 1024) dtype=float32>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)[0m

Arguments received by TFBertModel.call():
  • args=('<KerasTensor shape=(None, 34), dtype=int32, sparse=None, name=input_ids>',)
  • kwargs={'attention_mask': '<KerasTensor shape=(None, 34), dtype=int32, sparse=None, name=attention_mask>', 'training': 'False'}